In [ ]:
import requests
from bs4 import BeautifulSoup
headers={"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36"}

In [ ]:
import yaml
def get_book_list_in_page(index,get_books, get_book_name, get_book_author, headers):
    try:
        with open("book_list.yml") as f:
            cache = yaml.safe_load(f) or {}
            if cache and index in cache:
                return cache[index]
    except IOError:
        cache = {}
    if index in cache:
        return cache[index]
    response = requests.get(f"https://book.douban.com/top250?start={index}", headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    book_list = [( get_book_name(book), get_book_author(book) ) for book in get_books(soup)]
    with open("book_list.yml", "w") as f:
        cache[index] = book_list
        yaml.safe_dump(cache, f, allow_unicode=True)
    return book_list

In [ ]:
get_books = lambda soup: soup.find_all("tr", class_="item")
get_book_name = lambda book: book.find("a", title=True).get("title")
get_book_author = lambda book: book.find("p", class_="pl").text.split("/")[0].replace("著", "").strip()
flatten = lambda lst: [item for sublist in lst for item in sublist]
book_list = flatten( [get_book_list_in_page(index, get_books, get_book_name, get_book_author, headers) for index in range(1, 11)] )